In [86]:
import random
import textwrap
import openpyxl


def create_puzzle_table(words):
    # Find the maximum length of words
    max_len = max(len(word) for word in words)
    
    # Create a puzzle table
    puzzle = [[' ' for _ in range(max_len)] for _ in range(max_len)]
    
    # Place words horizontally or vertically randomly
    for word in words:
        direction = random.choice(['horizontal', 'vertical'])
        if direction == 'horizontal':
            place_word_horizontal(word, puzzle)
        else:
            place_word_vertical(word, puzzle)
    
    # Fill remaining spaces with random letters
    fill_random_letters(puzzle)
    
    return puzzle

def place_word_horizontal(word, puzzle):
    word_len = len(word)
    max_len = len(puzzle)
    row = random.randint(0, max_len - 1)
    col = random.randint(0, max_len - word_len)
    for i, letter in enumerate(word):
        puzzle[row][col + i] = letter

def place_word_vertical(word, puzzle):
    word_len = len(word)
    max_len = len(puzzle)
    row = random.randint(0, max_len - word_len)
    col = random.randint(0, max_len - 1)
    for i, letter in enumerate(word):
        puzzle[row + i][col] = letter

def fill_random_letters(puzzle):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i in range(len(puzzle)):
        for j in range(len(puzzle[i])):
            if puzzle[i][j] == ' ':
                puzzle[i][j] = random.choice(alphabet)

from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas

def display_puzzle(puzzle, words, tab_name):
    # Specify the DPI (dots per inch)
    dpi = 300

    # Calculate the page size based on DPI
    page_width = 8.5 * inch
    page_height = 11 * inch

    # Create a canvas object with the specified page size
    c = canvas.Canvas(f"{tab_name}.pdf", pagesize=(page_width, page_height))

    # Adjusted to account for 0.5-inch gap on each side
    usable_width = page_width - (0.5 * inch * 2)
    usable_height = page_height - (0.5 * inch * 2)

    # Calculate cell size based on the image dimensions and puzzle size
    cell_size = min(usable_width // len(puzzle), usable_height // len(puzzle[0]))

    # Calculate the offset to center the puzzle in the page, including the 0.5-inch gap
    x_offset = (page_width - cell_size * len(puzzle[0])) // 2
    y_offset = (page_height - cell_size * len(puzzle)) // 2

    # Ensure the offsets are at least 0.5 inch from the page edges
    x_offset = max(x_offset, 0.5 * inch)
    y_offset = max(y_offset, 0.5 * inch)


    # Draw the puzzle
    for i, row in enumerate(puzzle):
        for j, letter in enumerate(row):
            # Calculate coordinates for the current cell
            x0 = x_offset + j * cell_size
            y0 = y_offset + i * cell_size
            x1 = x0 + cell_size
            y1 = y0 + cell_size

            # Draw cell border
            c.rect(x0, y0, cell_size, cell_size)

            # Draw letter (capitalized)
            c.setFont("Helvetica", 10)
            c.drawString(x0 + cell_size // 3, y0 + cell_size // 3, letter.upper())

    # Draw a rectangular box around the puzzle table
    c.rect(x_offset, y_offset, cell_size * len(puzzle[0]), cell_size * len(puzzle))

    # Draw the list of words at the bottom with word wrapping
    bottom_text = ",".join(words)  # Join words with commas
    max_width = 70  # Adjust max width as needed
    wrapped_text = textwrap.wrap(bottom_text, width=max_width)  # Apply word wrapping
    c.setFont("Helvetica", 10)

    # Adjust starting y-coordinate for the wrapped text
    text_y_offset = y_offset - 20
    line_height = 12  # Adjust line height as needed

    for line in wrapped_text:
        c.drawString(x_offset, text_y_offset, line)
        text_y_offset -= line_height  # Move to the next line
    
    # Draw the heading with the tab name
    c.setFont("Helvetica", 16)
    c.drawString(x_offset, y_offset + page_height - 40, f"Tab Name: {tab_name}")

    # Save the canvas as a PDF file
    c.save()

    print(f"PDF file for '{tab_name}' saved successfully.")

def generate_pdf_from_excel(input_file):
    # Load the Excel workbook
    wb = openpyxl.load_workbook(input_file)

    # Loop through each sheet in the workbook
    for sheet_name in wb.sheetnames:
        # Extract data from the sheet and flatten the list
        sheet = wb[sheet_name]
        data = [str(cell) for row in sheet.iter_rows(values_only=True) for cell in row]

        # Create puzzle table
        puzzle = create_puzzle_table(data)

        # Join bottom words with comma
        bottom_words = ', '.join(data)

        # Display puzzle and generate PDF
        display_puzzle(puzzle, data, sheet_name)

def main():
    # Input Excel file
    input_file = 'Chemistryteks.xlsx'

    # Generate PDF from Excel
    generate_pdf_from_excel(input_file)

if __name__ == "__main__":
    main()


PDF file for 'periodic table' saved successfully.
PDF file for 'Atomic theory' saved successfully.
PDF file for 'Nomenclature' saved successfully.
PDF file for 'Chemical bonding' saved successfully.
PDF file for 'Mole concept' saved successfully.
PDF file for 'Chemical equations and reaction' saved successfully.
PDF file for 'Gas behavior' saved successfully.
PDF file for 'Behaviors of solution' saved successfully.
PDF file for 'acids and bases' saved successfully.
PDF file for 'Energy in chemical reactions' saved successfully.
PDF file for 'Nuclear chemistry' saved successfully.


In [96]:
import random
import textwrap
import openpyxl
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas

def create_puzzle_table(words):
    max_len = max(len(word) for word in words)
    puzzle = [[' ' for _ in range(max_len)] for _ in range(max_len)]
    for word in words:
        direction = random.choice(['horizontal', 'vertical'])
        if direction == 'horizontal':
            place_word_horizontal(word, puzzle)
        else:
            place_word_vertical(word, puzzle)
    fill_random_letters(puzzle)
    return puzzle

def place_word_horizontal(word, puzzle):
    word_len, max_len = len(word), len(puzzle)
    row, col = random.randint(0, max_len - 1), random.randint(0, max_len - word_len)
    for i, letter in enumerate(word):
        puzzle[row][col + i] = letter

def place_word_vertical(word, puzzle):
    word_len, max_len = len(word), len(puzzle)
    row, col = random.randint(0, max_len - word_len), random.randint(0, max_len - 1)
    for i, letter in enumerate(word):
        puzzle[row + i][col] = letter

def fill_random_letters(puzzle):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i in range(len(puzzle)):
        for j in range(len(puzzle[i])):
            if puzzle[i][j] == ' ':
                puzzle[i][j] = random.choice(alphabet)

def display_puzzle(puzzle, words, tab_name, headers):
    page_width, page_height = 8.5 * inch, 11 * inch
    c = canvas.Canvas(f"{tab_name}.pdf", pagesize=(page_width, page_height))
    usable_width, usable_height = page_width - (0.5 * inch * 2), page_height - (0.5 * inch * 2)
    cell_size = min(usable_width // len(puzzle), usable_height // len(puzzle[0]))
    x_offset = (page_width - cell_size * len(puzzle[0])) // 2
    y_offset = (page_height - cell_size * len(puzzle)) // 2
    x_offset, y_offset = max(x_offset, 0.5 * inch), max(y_offset, 0.5 * inch)

    # Use tab_name as the header
    c.setFont("Helvetica", 12)
    c.drawString(x_offset, page_height - 50, tab_name)  # tab_name is used as the header here

    # Draw the puzzle
    for i, row in enumerate(puzzle):
        for j, letter in enumerate(row):
            x0, y0 = x_offset + j * cell_size, y_offset + i * cell_size
            c.rect(x0, y0, cell_size, cell_size)
            c.setFont("Helvetica", 10)
            c.drawString(x0 + cell_size // 3, y0 + cell_size // 3, letter.upper())

    # Draw words at the bottom
    bottom_text = ", ".join(words)
    max_width = 70
    wrapped_text = textwrap.wrap(bottom_text, width=max_width)
    c.setFont("Helvetica", 10)
    text_y_offset = y_offset - 20
    line_height = 12
    for line in wrapped_text:
        c.drawString(x_offset, text_y_offset, line)
        text_y_offset -= line_height

    # Save the PDF
    c.save()
    print(f"PDF file for '{tab_name}' saved successfully.")


def generate_pdf_from_excel(input_file):
    wb = openpyxl.load_workbook(input_file)
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        # Extract headers from the first row, assuming the first row contains headers
        headers = [cell.value for cell in sheet[1]]

        # Extract data from the rest of the sheet, skipping the first row
        # Since values_only=True, you directly get cell values, not cell objects, so no need for .value
        data = [str(cell) for row in sheet.iter_rows(min_row=2, values_only=True) for cell in row if cell is not None]

        puzzle = create_puzzle_table(data)
        display_puzzle(puzzle, data, sheet_name, headers)

def main():
    input_file = 'Chemistryteks_4.xlsx'
    generate_pdf_from_excel(input_file)

if __name__ == "__main__":
    main()


PDF file for 'periodic table' saved successfully.
PDF file for 'Atomic theory' saved successfully.
PDF file for 'Nomenclature' saved successfully.
PDF file for 'Chemical bonding' saved successfully.
PDF file for 'Mole concept' saved successfully.
PDF file for 'Chemical equations and reaction' saved successfully.
PDF file for 'Gas behavior' saved successfully.
PDF file for 'Behaviors of solution' saved successfully.
PDF file for 'acids and bases' saved successfully.
PDF file for 'Energy in chemical reactions' saved successfully.
PDF file for 'Nuclear chemistry' saved successfully.


In [95]:
import random
import textwrap
import openpyxl
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas

def place_word_diagonal(word, puzzle):
    word_len, max_len = len(word), len(puzzle)
    # Ensure there's enough space for the word to fit diagonally
    start_max = max_len - word_len
    if start_max < 0: return  # Not enough space to place the word

    row, col = random.randint(0, start_max), random.randint(0, start_max)
    # Check if the diagonal from (row, col) is free for the word
    for i in range(word_len):
        if puzzle[row + i][col + i] != ' ' and puzzle[row + i][col + i] != word[i]:
            return  # Collision detected, abort placement

    # Place the word if the path is clear
    for i, letter in enumerate(word):
        puzzle[row + i][col + i] = letter

def create_puzzle_table(words):
    max_len = max(len(word) for word in words)
    puzzle = [[' ' for _ in range(max_len)] for _ in range(max_len)]
    word_positions = []  # To store word positions
    for word in words:
        direction = random.choice(['horizontal', 'vertical', 'diagonal'])
        positions = None
        if direction == 'horizontal':
            positions = place_word_horizontal(word, puzzle)
        elif direction == 'vertical':
            positions = place_word_vertical(word, puzzle)
        else:
            positions = place_word_diagonal(word, puzzle)
        if positions:
            word_positions.append(positions)
    fill_random_letters(puzzle)
    return puzzle, word_positions


def highlight_words(c, word_positions, cell_size, x_offset, y_offset):
    c.setFillGray(0.75)  # Light gray for highlighting
    for positions in word_positions:
        for pos in positions:
            row, col = pos
            x0, y0 = x_offset + col * cell_size, y_offset + row * cell_size
            c.rect(x0, y0, cell_size, cell_size, fill=1)
    c.setFillGray(0)  # Reset to black for text


def place_word_horizontal(word, puzzle):
    word_len, max_len = len(word), len(puzzle)
    row, col = random.randint(0, max_len - 1), random.randint(0, max_len - word_len)
    for i, letter in enumerate(word):
        puzzle[row][col + i] = letter
    return [(row, col + i) for i in range(word_len)]  # Return positions

def place_word_vertical(word, puzzle):
    word_len, max_len = len(word), len(puzzle)
    row, col = random.randint(0, max_len - word_len), random.randint(0, max_len - 1)
    for i, letter in enumerate(word):
        puzzle[row + i][col] = letter
    return [(row, col + i) for i in range(word_len)] 

def fill_random_letters(puzzle):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i in range(len(puzzle)):
        for j in range(len(puzzle[i])):
            if puzzle[i][j] == ' ':
                puzzle[i][j] = random.choice(alphabet)

def display_puzzle(puzzle, words, tab_name, headers,word_positions=None):
    page_width, page_height = 8.5 * inch, 11 * inch
    c = canvas.Canvas(f"{tab_name}.pdf", pagesize=(page_width, page_height))
    usable_width, usable_height = page_width - (0.5 * inch * 2), page_height - (0.5 * inch * 2)
    cell_size = min(usable_width // len(puzzle), usable_height // len(puzzle[0]))
    x_offset = (page_width - cell_size * len(puzzle[0])) // 2
    y_offset = (page_height - cell_size * len(puzzle)) // 2
    x_offset, y_offset = max(x_offset, 0.5 * inch), max(y_offset, 0.5 * inch)

    # Draw headers at the top
    c.setFont("Helvetica", 12)
    header_text = ", ".join(headers)
    c.drawString(x_offset, page_height - 50, header_text)  # Adjust the Y position as needed

    if word_positions:
      highlight_words(c, word_positions, cell_size, x_offset, y_offset)


    # Draw the puzzle
    for i, row in enumerate(puzzle):
        for j, letter in enumerate(row):
            x0, y0 = x_offset + j * cell_size, y_offset + i * cell_size
            c.rect(x0, y0, cell_size, cell_size)
            c.setFont("Helvetica", 10)
            c.drawString(x0 + cell_size // 3, y0 + cell_size // 3, letter.upper())

    # Draw words at the bottom
    bottom_text = ", ".join(words)
    max_width = 70
    wrapped_text = textwrap.wrap(bottom_text, width=max_width)
    c.setFont("Helvetica", 10)
    text_y_offset = y_offset - 20
    line_height = 12
    for line in wrapped_text:
        c.drawString(x_offset, text_y_offset, line)
        text_y_offset -= line_height

    # Save the PDF
    c.save()
    print(f"PDF file for '{tab_name}' saved successfully.")

def generate_pdf_from_excel(input_file):
    wb = openpyxl.load_workbook(input_file)
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        # Extract headers from the first row, assuming the first row contains headers
        headers = [cell.value for cell in sheet[1]]

        # Extract data from the rest of the sheet, skipping the first row
        # Since values_only=True, you directly get cell values, not cell objects, so no need for .value
        data = [str(cell) for row in sheet.iter_rows(min_row=2, values_only=True) for cell in row if cell is not None]

        puzzle = create_puzzle_table(data)
        display_puzzle(puzzle, data, sheet_name, headers)

def main():
    input_file = 'Chemistryteks_4.xlsx'
    generate_pdf_from_excel(input_file)

if __name__ == "__main__":
    main()


AttributeError: 'list' object has no attribute 'upper'

In [ ]:
def place_word_diagonal(word, puzzle):
    word_len, max_len = len(word), len(puzzle)
    # Ensure there's enough space for the word to fit diagonally
    start_max = max_len - word_len
    if start_max < 0: return  # Not enough space to place the word

    row, col = random.randint(0, start_max), random.randint(0, start_max)
    # Check if the diagonal from (row, col) is free for the word
    for i in range(word_len):
        if puzzle[row + i][col + i] != ' ' and puzzle[row + i][col + i] != word[i]:
            return  # Collision detected, abort placement

    # Place the word if the path is clear
    for i, letter in enumerate(word):
        puzzle[row + i][col + i] = letter

def create_puzzle_table(words):
    max_len = max(len(word) for word in words)
    puzzle = [[' ' for _ in range(max_len)] for _ in range(max_len)]
    for word in words:
        direction = random.choice(['horizontal', 'vertical', 'diagonal'])
        if direction == 'horizontal':
            place_word_horizontal(word, puzzle)
        elif direction == 'vertical':
            place_word_vertical(word, puzzle)
        else:
            place_word_diagonal(word, puzzle)
    fill_random_letters(puzzle)
    return puzzle
